In [171]:
import mlrose
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import time
import random

dir_out = '../plots/'

RANDOM_STATE_MODEL = 42


# TSP

In [172]:
# Create list of city coordinates
#length = 8
#coords_list = [(1, 1), (4, 2), (5, 2), (6, 4), (4, 4), (3, 6), (1, 5), (2, 3)]length = 20
length = 20
coords_list = np.random.random(size = (length, 2)) * 100
# Initialize fitness function object using coords_list
fitness_coords = mlrose.TravellingSales(coords = coords_list)

In [173]:
# Define optimization problem object
problem = 'TSP'
problem_fit = mlrose.TSPOpt(length = length, fitness_fn = fitness_coords, maximize=True)


In [174]:
# RHC
algorithm = 'RHC'
times_RHC = []
fitness_curves_RHC = []
fitness_RHC = []

time_start = time.time()
mutation_prob = 0.1
best_state, best_fitness, fitness_curve = mlrose.random_hill_climb(problem_fit,
                    max_attempts=100, # default 10
                    max_iters = 3000, # default np.inf
                    restarts=0, # default
                    curve = True, # default False
                    init_state=None, # default
                    random_state = RANDOM_STATE_MODEL # default None
                    )

times_RHC.append(time.time() - time_start)
fitness_curves_RHC.append(fitness_curve)
fitness_RHC.append(best_fitness)



In [175]:
plt.rcParams["figure.figsize"] = (8, 8)
plt.rcParams.update({'font.size': 20})

for i in range(len(times_RHC)):
    plt.plot(fitness_curves_RHC[i])
    
plt.title(problem + ' by ' + algorithm + ': Iteration-Fitness', fontsize = 25)
plt.xlabel('Iterations')
plt.ylabel('Fitness')
plt.grid()
plt.legend()
plt.tight_layout()
plt.savefig(dir_out + problem +'_' + algorithm + '_Fitness.png')
plt.close()


#####################3
df_RHC = pd.DataFrame()
df_RHC['Optimized Fitness'] = fitness_RHC
df_RHC['Trainning Time'] = times_RHC

df_RHC

No handles with labels found to put in legend.


,Optimized Fitness,Trainning Time
0,1460.773059,0.07928


In [176]:
# SA
algorithm = 'SA'
GeomDecay = mlrose.GeomDecay(init_temp=1.0, decay=0.99, min_temp=0.001)
ArithDecay = mlrose.ArithDecay(init_temp=1.0, decay=0.0001, min_temp=0.001)
ExpDecay = mlrose.ExpDecay(init_temp=1.0, exp_const=0.005, min_temp=0.001)

schedules = [GeomDecay, ArithDecay, ExpDecay]
schedule_names = ['GeomDecay', 'ArithDecay', 'ExpDecay' ]
times_SA = []
fitness_curves_SA = []
fitness_SA = []

for i in range(len(schedules)):
    schedule = schedules[i]
    schedule_name = schedule_names[i]
    
    time_start = time.time()
    best_state, best_fitness, fitness_curve = mlrose.simulated_annealing(
                    problem_fit,
                    schedule = schedule,
                    max_attempts = 100, # default 10
                    max_iters = 3000, # default np.inf
                    init_state=None, # default
                    curve = True, # default False
                    random_state = RANDOM_STATE_MODEL # default None
                    )
        
    times_SA.append(time.time() - time_start)
    fitness_curves_SA.append(fitness_curve)
    fitness_SA.append(best_fitness)


In [177]:
plt.rcParams["figure.figsize"] = (8, 8)
plt.rcParams.update({'font.size': 20})

for i in range(len(schedules)):
    schedule_name = schedule_names[i]
    plt.plot(fitness_curves_SA[i], label = schedule_name)
    
plt.title(problem + ' by ' + algorithm + ': Iteration-Fitness', fontsize = 25)
plt.xlabel('Iterations')
plt.ylabel('Fitness')
#plt.xlim(0,3000)
plt.grid()
plt.legend()
plt.tight_layout()
plt.savefig(dir_out + problem +'_' + algorithm + '_Fitness.png')
plt.close()


#####################3
df_SA = pd.DataFrame()
df_SA['Schedule'] = schedule_names
df_SA['Optimized Fitness'] = fitness_SA
df_SA['Trainning Time'] = times_SA

df_SA

,Schedule,Optimized Fitness,Trainning Time
0,GeomDecay,1469.524362,0.127405
1,ArithDecay,1469.365149,0.234298
2,ExpDecay,1469.233847,0.130397


In [178]:
# GA
algorithm = 'GA'
mutation_probs = [0.1,0.5,0.7,0.9]
times_GA = []
fitness_curves_GA = []
fitness_GA = []
for mutation_prob in mutation_probs:
    time_start = time.time()
    best_state, best_fitness, fitness_curve = mlrose.genetic_alg(
                       problem_fit,
                       pop_size = 100, # default 200
                       mutation_prob = mutation_prob, # default 0.1
                       max_attempts = 100, # default 10
                       max_iters = 3000, # default np.inf
                       curve = True, # default False
                       random_state = RANDOM_STATE_MODEL # default None
                       )
    
    times_GA.append(time.time() - time_start)
    fitness_curves_GA.append(fitness_curve)
    fitness_GA.append(best_fitness)


In [179]:
plt.rcParams["figure.figsize"] = (8, 8)
plt.rcParams.update({'font.size': 20})

for i in range(len(mutation_probs)):
    mutation_prob = mutation_probs[i]
    plt.plot(fitness_curves_GA[i], label = 'Mutation Prob: '+ str(mutation_prob))
    
plt.title(problem + ' by ' + algorithm + ': Iteration-Fitness', fontsize = 25)
plt.xlabel('Iterations')
plt.ylabel('Fitness')
plt.grid()
plt.legend()
plt.tight_layout()
plt.savefig(dir_out + problem +'_' + algorithm + '_Fitness.png')
plt.close()


#####################3
df_GA = pd.DataFrame()
df_GA['Mutation Prob'] = mutation_probs
df_GA['Optimized Fitness'] = fitness_GA
df_GA['Trainning Time'] = times_GA

df_GA

,Mutation Prob,Optimized Fitness,Trainning Time
0,0.1,1408.212009,13.638072
1,0.5,1408.873455,4.594693
2,0.7,1414.439744,8.960223
3,0.9,1404.503502,4.573011


In [180]:
# MIMIC
algorithm = 'MIMIC'
keep_pcts = [0.1,0.5]
times_MIMIC = []
fitness_curves_MIMIC = []
fitness_MIMIC = []
for keep_pct in keep_pcts:
    time_start = time.time()
    best_state, best_fitness, fitness_curve = mlrose.mimic(
                        problem_fit,
                        pop_size = 200, # default 200
                        keep_pct = keep_pct, # default 0.2
                        max_attempts = 100, # default 10
                        max_iters = 3000, # default np.inf
                        curve = True, # default False
                        random_state = RANDOM_STATE_MODEL, # default None
                        fast_mimic=False # detault False
                        )
    
    
    times_MIMIC.append(time.time() - time_start)
    fitness_curves_MIMIC.append(fitness_curve)
    fitness_MIMIC.append(best_fitness)


In [181]:
plt.rcParams["figure.figsize"] = (8, 8)
plt.rcParams.update({'font.size': 20})

for i in range(len(keep_pcts)):
    keep_pct = keep_pcts[i]
    plt.plot(fitness_curves_MIMIC[i], label = 'Keep Proportion: '+ str(keep_pct))
    
plt.title(problem + ' by ' + algorithm + ': Iteration-Fitness', fontsize = 25)
plt.xlabel('Iterations')
plt.ylabel('Fitness')
plt.grid()
plt.legend()
plt.tight_layout()
plt.savefig(dir_out + problem +'_' + algorithm + '_Fitness.png')
plt.close()


#####################3
df_MIMIC = pd.DataFrame()
df_MIMIC['Keep Proportion'] = keep_pcts
df_MIMIC['Optimized Fitness'] = fitness_MIMIC
df_MIMIC['Trainning Time'] = times_MIMIC

df_MIMIC

,Keep Proportion,Optimized Fitness,Trainning Time
0,0.1,1399.902288,48.544960
1,0.5,1410.265946,99.938219


# Compare Fitness and Time

In [182]:
df_Stat = pd.DataFrame(index = ['RHC', 'SA', 'GA','MIMIC'])
df_Stat['Optimized Fitness'] = [np.max(fitness_RHC), np.max(fitness_SA), np.max(fitness_GA), np.max(fitness_MIMIC)]
df_Stat['Trainning Time'] = [np.mean(times_RHC), np.mean(times_SA), np.mean(times_GA), np.mean(times_MIMIC)]
df_Stat

,Optimized Fitness,Trainning Time
RHC,1460.773059,0.079280
SA,1469.524362,0.164033
GA,1414.439744,7.941500
MIMIC,1410.265946,74.241590


In [183]:
plt.rcParams["figure.figsize"] = (9, 6)
plt.rcParams.update({'font.size': 20})

x_plot = df_Stat.index.values
y_plot = df_Stat['Optimized Fitness'].values
plt.bar(x_plot, y_plot)

values = y_plot
 
# Text on the top of each barplot
for i in range(len(df_Stat)):
    plt.text(x = x_plot[i] , y = y_plot[i]+np.max(y_plot)*0.01, s = "%.4f" % values[i], size = 20)
    
plt.title(problem + ': Fitness Comparison', fontsize = 25)
plt.xlabel('Algorithm')
plt.ylabel('Fitness')
plt.ylim(0,np.max(y_plot)*1.1)
plt.grid()
plt.tight_layout()
plt.savefig(dir_out + problem + '_Compare_Fitness.png')
plt.close()

In [184]:
plt.rcParams["figure.figsize"] = (9, 6)
plt.rcParams.update({'font.size': 20})

x_plot = df_Stat.index.values
y_plot = df_Stat['Trainning Time'].values
plt.bar(x_plot, y_plot)

values = y_plot
 
# Text on the top of each barplot
for i in range(len(df_Stat)):
    plt.text(x = x_plot[i] , y = y_plot[i]+np.max(y_plot)*0.01, s = "%.4f" % values[i], size = 20)
    
plt.title(problem + ': Training Time Comparison', fontsize = 25)
plt.xlabel('Algorithm')
plt.ylabel('Time / s')
plt.ylim(0,np.max(y_plot)*1.1)
plt.grid()
plt.tight_layout()
plt.savefig(dir_out +problem+'_Compare_Time.png')
plt.close()